In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
train_data =pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X_test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
sample_submission=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
y_train_all = train_data.pop('label')
X_train_all = train_data.copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_all , y_train_all , test_size=0.1 , random_state=40 )

In [ ]:
import tensorflow as tf 
from tensorflow import keras
IMG_SIZE= 28
X_train = X_train /255.0
X_valid = X_valid /255.0

y_train = y_train.values
y_valid = y_valid.values

X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_valid = X_valid.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
        min_delta=0.01, patience=10,   restore_best_weights=True)
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3 , beta_1=0.9 , beta_2=0.999)


In [ ]:
cnn = keras.models.Sequential([
    keras.layers.Conv2D(filters=64,kernel_size =  7, strides=1 ,padding="same", activation='relu',input_shape=X_train.shape[1:]),
    keras.layers.Conv2D(filters=64,kernel_size =  7, strides=1 ,padding="same", activation='relu',input_shape=X_train.shape[1:]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Dropout(0.15),

    
        
    keras.layers.Conv2D(filters=128,kernel_size =  3, strides=1 ,padding="same" ,activation='relu'),
    keras.layers.Conv2D(filters=128,kernel_size =  3, strides=1 ,padding="same" ,activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.15),

    
    keras.layers.Conv2D(filters=256,kernel_size =  3 ,strides=1, padding="same" ,activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.15),


    keras.layers.Flatten(),

    keras.layers.Dense(265, activation="relu"),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(10, activation='softmax')
])
cnn.compile(loss ='sparse_categorical_crossentropy', optimizer = opt , metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.1, 
        height_shift_range=0.1, 
)
datagen.fit(X_train)

In [ ]:
cnn.fit_generator(datagen.flow(X_train,y_train, batch_size=36), validation_data = (X_valid,y_valid),
                           epochs = 100   , callbacks=[early_stopping])

In [ ]:
X_test = X_test.values /255
X_test_reshapes = X_test.reshape(-1, IMG_SIZE, IMG_SIZE,1)
result = np.argmax(cnn.predict(X_test_reshapes),axis=-1)

In [ ]:
submission_df = pd.DataFrame({'ImageId':sample_submission['ImageId'],'Label':result})
submission_df.to_csv('submission.csv',index=False)